In [1]:
import os
import json
import pandas as pd

# Data is in the data directory in project_1 folder
DATA_DIR_MOV = os.path.join('data', 'boxofficemojo')

# 'listdir' function from 'os'  gives you a list of all
target_file_names_Mov = os.listdir(DATA_DIR_MOV)

movie = list()
for files in target_file_names_Mov:
    target_file_path = os.path.join(DATA_DIR_MOV, files)
    with open(target_file_path,'r') as target_file:
        movie.append(json.load(target_file))
        

movies_df = pd.DataFrame(movie)        

In [2]:
len(movies_df)

3728

In [3]:
DATA_DIR_CRI = os.path.join('data', 'metacritic')

# 'listdir' function from 'os'  gives you a list of all
target_file_names_Cri = os.listdir(DATA_DIR_CRI)

critics = list()
for files in target_file_names_Cri:
    target_file_path = os.path.join(DATA_DIR_CRI, files)
    with open(target_file_path,'r') as target_file:
        critics.append(json.load(target_file))
        

In [5]:
list_key=critics[1].keys()

count = 0            
for i,crt in enumerate(critics):
    try:
        if(list_key != crt.keys()):
            count += 1
            print i,crt
    except AttributeError:
        count=count
        print i,crt

print count

1251 {u'unable to retrieve': []}
4439 [u'/movie/11-flowers']
4765 {u'unable to retrieve': [u'/movie/once-upon-a-time-in-mexico', u'/movie/serpico']}
4766 []
2


In [7]:
# Removing Attribute Error rows which is not a dictionary
# Removing rows which doesn't have all keys
    
list_index = [1251,4439,4765,4766]

critics = [j for i,j in enumerate(critics) if i not in list_index]

In [8]:
list_key=critics[1].keys()

count = 0            
for i,crt in enumerate(critics):
    try:
        if(list_key != crt.keys()):
            count += 1
            print i,crt
    except AttributeError:
        count=count
        print i,crt

print count

0


In [9]:
# Creating Genre list

genre_list = []

for crt in critics:
    try:
        for i in crt['genre']:
            genre_list.append(i)
    except KeyError:
        print "Key Error"

In [10]:
len(genre_list)

12838

In [11]:
# Finding unique genre

unique_genre = set(genre_list)

In [12]:
len(unique_genre)

25

In [13]:
# Creating dummy variables for each reviews ofr user as well as critic
# Also, addded percentage columns

for i,crt in enumerate(critics):
    try:
        crt['critic_positive'] = crt['num_critic_reviews'][0]
        crt['critic_netural'] = crt['num_critic_reviews'][1]
        crt['critic_negative'] = crt['num_critic_reviews'][2]
        crt['critic_total'] = crt['num_critic_reviews'][3]
        
        crt['user_positive'] = crt['num_user_reviews'][0]
        crt['user_netural'] = crt['num_user_reviews'][1]
        crt['user_negative'] = crt['num_user_reviews'][2]
        crt['user_total'] = crt['num_user_reviews'][3]

        if crt['num_critic_reviews'][3] !=0:
            crt['critic_positive_pct'] = float(crt['num_critic_reviews'][0])/crt['num_critic_reviews'][3]
            crt['critic_netural_pct'] = float(crt['num_critic_reviews'][1])/crt['num_critic_reviews'][3]
            crt['critic_negative_pct'] = float(crt['num_critic_reviews'][2])/crt['num_critic_reviews'][3]
        else:
            crt['critic_positive_pct'] = 0.0
            crt['critic_netural_pct'] = 0.0
            crt['critic_negative_pct'] = 0.0
            
        if crt['num_user_reviews'][3] !=0:
            crt['user_positive_pct'] = float(crt['num_user_reviews'][0])/crt['num_user_reviews'][3]
            crt['user_netural_pct'] = float(crt['num_user_reviews'][1])/crt['num_user_reviews'][3]
            crt['user_negative_pct'] = float(crt['num_user_reviews'][2])/crt['num_user_reviews'][3]
        else:
            crt['user_positive_pct'] = 0.0
            crt['user_netural_pct'] = 0.0
            crt['user_negative_pct'] = 0.0    
    
    except KeyError:
        print i,"Key Error"

In [14]:
# Adding columns for each genre and making it 1 if the movie belongs to that genre

for i,crt in enumerate(critics):
    try:
        for gr in unique_genre:
            if gr in crt['genre']:
                crt[gr]=1
            else:
                crt[gr]=0
    except KeyError:
        print "Key Error"
            

In [15]:
critics_df=pd.DataFrame(critics)

In [16]:
critics_df.shape

(4763, 54)

In [17]:
# Creating cleaned title with only character and number

movies_df['cleaned_title'] = movies_df['title'].str.lower()
movies_df['cleaned_title'] = movies_df['cleaned_title'].str.replace("\W",'')

critics_df['cleaned_title'] = critics_df['title'].str.lower()
critics_df['cleaned_title'] = critics_df['cleaned_title'].str.replace("\W",'')

In [18]:
# Joining both the data on cleaned title

movie_critics_df = pd.merge(movies_df, critics_df, on='cleaned_title', how='inner')

In [19]:
movie_critics_df.shape

(3513, 68)

In [20]:
# Mismatch data from movies_df

len(movies_df[~movies_df.cleaned_title.isin(movie_critics_df.cleaned_title)])

242